In [ ]:
# Modelo simple  para COVID con inmunidad temporal

#Este modelo es capaz de modelar una simulacion con las siguientes caacteristicas
import numpy as np
from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt



In [ ]:
# ----------------- Estados -----------------
EMPTY = 0      # espacio vacío (sin persona) sitios comunes altamente concurrido
#no representa un sitio abierto mas bien un punto en comun que puede funcionar como vector,
# como perillas, barandas,cajeros, etc
SUS = 1        # susceptible, persona comun
INF = 2        # infectado (puede contagiar)
REC = 3        # recuperado (inmune temporalmente)
DEAD = 4       # fallecido (no contagia)

In [ ]:
# ----------------- Parámetros (ajusta aquí) -----------------
SIZE = 100                   # tamaño del grid (SIZE x SIZE)
person_density = 0.55         # fracción de celdas ocupadas por personas
init_infected_frac = 0.01    # fracción inicial de infectados (sobre población)
infectious_period = 7        # días que una persona permanece infectiva
mortality_prob = 0.025        # probabilidad de morir al terminar la infección (2%)
p_trans_no_mask = 0.05       # prob. diaria de transmisión por vecino infectado (sin mascarilla)
MASK_EFFECT = 0.5            # factor multiplicador si la persona usa mascarilla (reduce p a la mitad)
mask_coverage = 0.0          # fracción de personas que usan mascarilla (0..1). Pon 0 si no usas máscaras.
p_background = 0.03          # probabilidad de infección por día independiente (espacio / comunidad) = 3%


In [ ]:
# ------- Nueva regla: inmunidad temporal -------
immunity_duration = 28       # días que dura la inmunidad tras REC; luego vuelve a SUS


In [ ]:
# ----------------- Inicialización -----------------
np.random.seed(42)

In [ ]:
def initialize_grid(size, density=0.7, init_infected_frac=0.01, mask_cov=0.0):
    grid = np.zeros((size, size), dtype=int)       # estado
    timer = np.zeros((size, size), dtype=int)      # cuenta días infectivo (si INF) o días desde recuperación (si REC)
    uses_mask = np.zeros((size, size), dtype=bool) # si la persona usa mascarilla

    # seleccionar celdas ocupadas por personas
    indices = [(i,j) for i in range(size) for j in range(size)]
    n_people = int(size*size * density)
    chosen = np.random.choice(len(indices), size=n_people, replace=False)
    chosen_coords = [indices[k] for k in chosen]

    # poner susceptibles
    for (i,j) in chosen_coords:
        grid[i,j] = SUS

    # asignar mascarillas según cobertura
    if mask_cov > 0:
        n_mask = int(n_people * mask_cov)
        mask_idxs = np.random.choice(n_people, size=n_mask, replace=False)
        for idx in mask_idxs:
            i,j = chosen_coords[idx]
            uses_mask[i,j] = True

    # asignar algunos infectados iniciales (los dejamos INF desde el inicio)
    n_init_inf = max(1, int(n_people * init_infected_frac))
    inf_idxs = np.random.choice(n_people, size=n_init_inf, replace=False)
    for idx in inf_idxs:
        i,j = chosen_coords[idx]
        grid[i,j] = INF
        timer[i,j] = infectious_period

    return grid, timer, uses_mask


In [ ]:
# ----------------- Vecinos (Moore r=1) -----------------
def neighbors(i,j,size):
    for di in [-1,0,1]:
        for dj in [-1,0,1]:
            if di==0 and dj==0:
                continue
            ni, nj = i+di, j+dj
            if 0 <= ni < size and 0 <= nj < size:
                yield ni, nj

In [ ]:
# ----------------- Regla de actualización (con inmunidad temporal) -----------------
def update_simple_with_waning(grid, timer, uses_mask,
                  p_trans_no_mask=0.05,
                  mask_effect=0.5,
                  p_background=0.03,
                  infectious_period=7,
                  mortality_prob=0.02,
                  immunity_duration=30):
    size = grid.shape[0]
    new_grid = grid.copy()
    new_timer = timer.copy()

    for i in range(size):
        for j in range(size):
            state = grid[i,j]

            # SUSCEPTIBLE: revisar contagio por vecinos y por fondo
            if state == SUS:
                prob_no_inf = 1.0

                # contagio por vecinos infectados
                for (ni,nj) in neighbors(i,j,size):
                    if grid[ni,nj] == INF:
                        # calcular probabilidad efectiva por ese vecino
                        p = p_trans_no_mask
                        # si la persona susceptible usa mascarilla, reducir entrada
                        if uses_mask[i,j]:
                            p *= mask_effect
                        # si el vecino infectado usa mascarilla, reducir salida
                        if uses_mask[ni,nj]:
                            p *= mask_effect
                        # multiplicar probabilidad de no infectarse por este vecino
                        prob_no_inf *= (1.0 - p)
                # contagio por riesgo de fondo (espacio/comunidad)
                prob_no_inf *= (1.0 - p_background)

                p_infect = 1.0 - prob_no_inf
                if np.random.rand() < p_infect:
                    new_grid[i,j] = INF
                    new_timer[i,j] = infectious_period

            # INFECTADO: decrementar timer; si finaliza decidir REC o DEAD
            elif state == INF:
                new_timer[i,j] -= 1
                if new_timer[i,j] <= 0:
                    if np.random.rand() < mortality_prob:
                        new_grid[i,j] = DEAD
                        new_timer[i,j] = 0
                    else:
                        new_grid[i,j] = REC
                        new_timer[i,j] = 0  # reiniciar contador; ahora contará días desde recuperación

            # RECUPERADO: contar días desde recuperación; si excede immunity_duration -> volver a SUS
            elif state == REC:
                new_timer[i,j] += 1
                if new_timer[i,j] >= immunity_duration:
                    new_grid[i,j] = SUS
                    new_timer[i,j] = 0

            # DEAD o EMPTY: no cambian
            else:
                pass

    return new_grid, new_timer

In [ ]:
# ----------------- Animación simple -----------------
def animate_covid_with_waning(size=SIZE, steps=150, pause=0.08):
    grid, timer, uses_mask = initialize_grid(size, person_density, init_infected_frac, mask_coverage)
    colores = ['#FFFFFF', '#2ca02c', '#ff4500', '#1f77b4', '#000000']
    # (EMPTY, SUS, INF, REC, DEAD)
    cmap = ListedColormap(colores)

    plt.figure(figsize=(7,7))
    for t in range(steps):
        plt.clf()
        plt.imshow(grid, cmap=cmap, vmin=0, vmax=4)
        plt.title(f"COVID (modelo simple con inmunidad temporal) - Paso {t+1}/{steps}")
        cbar = plt.colorbar(ticks=[0,1,2,3,4])
        cbar.ax.set_yticklabels(['Empty','Sus','Inf','Rec','Dead'])
        plt.draw()
        plt.pause(pause)

        grid, timer = update_simple_with_waning(grid, timer, uses_mask,
                                    p_trans_no_mask=p_trans_no_mask,
                                    mask_effect=MASK_EFFECT,
                                    p_background=p_background,
                                    infectious_period=infectious_period,
                                    mortality_prob=mortality_prob,
                                    immunity_duration=immunity_duration)

    plt.show()

In [ ]:
# Ejecutar animación
animate_covid_with_waning()

Que representa?

En muchas ciudades se observaron picos de contagios muy altos, seguidos por un descenso abrupto. Ese descenso coincidía con una combinación de factores: una parte grande de la población ya estaba temporalmente inmune (por infección reciente o vacunación), cambios de comportamiento (uso de tapabocas, restricciones), y a veces estacionalidad.

Durante esas fases de descenso, la incidencia era baja y la gente sentía una “falsa seguridad”, como si se hubiera alcanzado inmunidad colectiva.

Sin embargo, la inmunidad decayó, aparecieron nuevas variantes más transmisibles (Delta, Ómicron) y se relajaron las medidas sociales. Esto volvió a abrir espacio para la transmisión y generó una nueva ola.

Por qué son “palpitantes” y no estables:

La inmunidad adquirida (por infección o vacuna) no era permanente ni homogénea en toda la población.

Cada nueva ola encontraba todavía grupos suficientes de susceptibles para reactivar la epidemia.

El umbral teórico de inmunidad de rebaño se “movía” hacia arriba con cada variante más contagiosa (subida del
𝑅_0).

Resultado: un patrón de picos repetidos (ola → descenso → nueva ola).

En ciudades como Madrid, Londres o Nueva York se vieron varias olas sucesivas (marzo 2020, invierno 2020–21, Delta en 2021, Ómicron en 2021–22). Cada una seguía el mismo patrón de “sube muchísimo → baja → vuelve a subir”.

Modelo diseñado utilizando ChatGPT5

Prompt:
"
Suponga una enfermedad, o un incendio forestal, o una moda, desarrolle un modelo de difusión usando ACs probabilísticos. O simule un robot con dos ruedas que evite obstáculos.

quiero empezar a diseñar un modelo simplificado del COVID, voy a hacerlo en un ipynb, primero definamos las reglas y asi vamos paso a paso

P_sin_T = 0 # persona sin tapabocas, con probabilidad de contagio de persona normal P_con_T = 1 # persona con tapabocas con prob de contagio de persona con tapabocas ESPACIO = A # Distancia entre personas, lo que impide contagio CONTAGIADO = B # Puede Contagiar TRATADO = #en aislamiento, contagia poco NO_TRATADO= #no aislamiento, puede contagiar FALLECIDO= # No contagia RECUPERADO= #No se puede contagiar en un tiempo y asi podemos empezar con un modelo bastante simplificado sin tapabocas cuanto era el contagio, con tapabocas cuanto era el contagio, indice de mortalidad. con un porcentaje de infectado"


>como puedo diseñarr que el modelo no ignore que la persona puede infectar durante varios dias a su conjunto cercano?, usando 30 dias para volver a ser susceptible
